# Task 1

In [123]:
route = ["NYC-LAX", "LHR-JFK", "SFO-SEA", "DXB-SIN", "MAD-ROM"]
day = ["Mon", "Tue", "Wed", "Thu", "Fri", "Sat", "Sun"]
class_1 = ["economy", "premium", "business"]
class_adj = [0, 80, 220]
route_adj = [140, 220, 60, 180, 80]
noise = rng.normal(0, 25)
tickets = []
for i in range(1,n+1):
    ticket = {
    'ticket_id': f"T{seed_value}-{i:04d}",
    'route' : route[(i+seed_value)%5],
    'day': day[(i+seed_value)%7],
    "days_to_departure": 1 + ((i * 3 + seed_value) % 60),
    "class": class_1[(i * 2 + seed_value) % 3],
    "price_usd" : round(120 + ((1 + (i * 3 + seed_value) % 60) * -1.5)+route_adj[(i+seed_value)%5]+class_adj[(i * 2 + seed_value) % 3]+rng.normal(0, 25),2)
    }
    if i%28==0:
        ticket["price_usd"] = ""
    if i%45==0:
        ticket["price_usd"] = ticket["price_usd"]*-1
    if i%37 == 0:
        ticket["class"] = ticket["class"].upper()
    tickets.append(ticket)
tickets

[{'ticket_id': 'T1610-0001',
  'route': 'LHR-JFK',
  'day': 'Tue',
  'days_to_departure': 54,
  'class': 'premium',
  'price_usd': 310.22},
 {'ticket_id': 'T1610-0002',
  'route': 'SFO-SEA',
  'day': 'Wed',
  'days_to_departure': 57,
  'class': 'economy',
  'price_usd': 113.65},
 {'ticket_id': 'T1610-0003',
  'route': 'DXB-SIN',
  'day': 'Thu',
  'days_to_departure': 60,
  'class': 'business',
  'price_usd': 395.69},
 {'ticket_id': 'T1610-0004',
  'route': 'MAD-ROM',
  'day': 'Fri',
  'days_to_departure': 3,
  'class': 'premium',
  'price_usd': 299.86},
 {'ticket_id': 'T1610-0005',
  'route': 'NYC-LAX',
  'day': 'Sat',
  'days_to_departure': 6,
  'class': 'economy',
  'price_usd': 249.95},
 {'ticket_id': 'T1610-0006',
  'route': 'LHR-JFK',
  'day': 'Sun',
  'days_to_departure': 9,
  'class': 'business',
  'price_usd': 550.82},
 {'ticket_id': 'T1610-0007',
  'route': 'SFO-SEA',
  'day': 'Mon',
  'days_to_departure': 12,
  'class': 'premium',
  'price_usd': 238.9},
 {'ticket_id': 'T1610-

In [151]:
len(tickets)

320

In [124]:
tickets[:5]

[{'ticket_id': 'T1610-0001',
  'route': 'LHR-JFK',
  'day': 'Tue',
  'days_to_departure': 54,
  'class': 'premium',
  'price_usd': 310.22},
 {'ticket_id': 'T1610-0002',
  'route': 'SFO-SEA',
  'day': 'Wed',
  'days_to_departure': 57,
  'class': 'economy',
  'price_usd': 113.65},
 {'ticket_id': 'T1610-0003',
  'route': 'DXB-SIN',
  'day': 'Thu',
  'days_to_departure': 60,
  'class': 'business',
  'price_usd': 395.69},
 {'ticket_id': 'T1610-0004',
  'route': 'MAD-ROM',
  'day': 'Fri',
  'days_to_departure': 3,
  'class': 'premium',
  'price_usd': 299.86},
 {'ticket_id': 'T1610-0005',
  'route': 'NYC-LAX',
  'day': 'Sat',
  'days_to_departure': 6,
  'class': 'economy',
  'price_usd': 249.95}]

# Task 2

In [183]:
cleaned_ticket = tickets.copy()

In [184]:
not_normal = []
for i in cleaned_ticket:
    if i["price_usd"]=="" or i["price_usd"]<0:
        not_normal.append(i)
not_normal

[{'ticket_id': 'T1610-0028',
  'route': 'DXB-SIN',
  'day': 'Mon',
  'days_to_departure': 15,
  'class': 'premium',
  'price_usd': ''},
 {'ticket_id': 'T1610-0045',
  'route': 'NYC-LAX',
  'day': 'Thu',
  'days_to_departure': 6,
  'class': 'business',
  'price_usd': -502.24},
 {'ticket_id': 'T1610-0056',
  'route': 'LHR-JFK',
  'day': 'Mon',
  'days_to_departure': 39,
  'class': 'economy',
  'price_usd': ''},
 {'ticket_id': 'T1610-0084',
  'route': 'MAD-ROM',
  'day': 'Mon',
  'days_to_departure': 3,
  'class': 'business',
  'price_usd': ''},
 {'ticket_id': 'T1610-0090',
  'route': 'NYC-LAX',
  'day': 'Sun',
  'days_to_departure': 21,
  'class': 'business',
  'price_usd': -427.17},
 {'ticket_id': 'T1610-0112',
  'route': 'SFO-SEA',
  'day': 'Mon',
  'days_to_departure': 27,
  'class': 'premium',
  'price_usd': ''},
 {'ticket_id': 'T1610-0135',
  'route': 'NYC-LAX',
  'day': 'Wed',
  'days_to_departure': 36,
  'class': 'business',
  'price_usd': -449.94},
 {'ticket_id': 'T1610-0140',
  

In [185]:
len(not_normal)

18

In [186]:
cleaned_tickets = [t for t in cleaned_ticket if t not in not_normal]
cleaned_tickets

[{'ticket_id': 'T1610-0001',
  'route': 'LHR-JFK',
  'day': 'Tue',
  'days_to_departure': 54,
  'class': 'premium',
  'price_usd': 310.22},
 {'ticket_id': 'T1610-0002',
  'route': 'SFO-SEA',
  'day': 'Wed',
  'days_to_departure': 57,
  'class': 'economy',
  'price_usd': 113.65},
 {'ticket_id': 'T1610-0003',
  'route': 'DXB-SIN',
  'day': 'Thu',
  'days_to_departure': 60,
  'class': 'business',
  'price_usd': 395.69},
 {'ticket_id': 'T1610-0004',
  'route': 'MAD-ROM',
  'day': 'Fri',
  'days_to_departure': 3,
  'class': 'premium',
  'price_usd': 299.86},
 {'ticket_id': 'T1610-0005',
  'route': 'NYC-LAX',
  'day': 'Sat',
  'days_to_departure': 6,
  'class': 'economy',
  'price_usd': 249.95},
 {'ticket_id': 'T1610-0006',
  'route': 'LHR-JFK',
  'day': 'Sun',
  'days_to_departure': 9,
  'class': 'business',
  'price_usd': 550.82},
 {'ticket_id': 'T1610-0007',
  'route': 'SFO-SEA',
  'day': 'Mon',
  'days_to_departure': 12,
  'class': 'premium',
  'price_usd': 238.9},
 {'ticket_id': 'T1610-

In [187]:
for i in cleaned_tickets:
    if i["class"] == i["class"].upper():
        i["class"]=i["class"].lower()
cleaned_tickets

[{'ticket_id': 'T1610-0001',
  'route': 'LHR-JFK',
  'day': 'Tue',
  'days_to_departure': 54,
  'class': 'premium',
  'price_usd': 310.22},
 {'ticket_id': 'T1610-0002',
  'route': 'SFO-SEA',
  'day': 'Wed',
  'days_to_departure': 57,
  'class': 'economy',
  'price_usd': 113.65},
 {'ticket_id': 'T1610-0003',
  'route': 'DXB-SIN',
  'day': 'Thu',
  'days_to_departure': 60,
  'class': 'business',
  'price_usd': 395.69},
 {'ticket_id': 'T1610-0004',
  'route': 'MAD-ROM',
  'day': 'Fri',
  'days_to_departure': 3,
  'class': 'premium',
  'price_usd': 299.86},
 {'ticket_id': 'T1610-0005',
  'route': 'NYC-LAX',
  'day': 'Sat',
  'days_to_departure': 6,
  'class': 'economy',
  'price_usd': 249.95},
 {'ticket_id': 'T1610-0006',
  'route': 'LHR-JFK',
  'day': 'Sun',
  'days_to_departure': 9,
  'class': 'business',
  'price_usd': 550.82},
 {'ticket_id': 'T1610-0007',
  'route': 'SFO-SEA',
  'day': 'Mon',
  'days_to_departure': 12,
  'class': 'premium',
  'price_usd': 238.9},
 {'ticket_id': 'T1610-

In [188]:
len(cleaned_tickets)

302

In [189]:
len(tickets)

320

# Task 3

In [190]:
price = np.array([i["price_usd"] for i in cleaned_tickets])

In [191]:
price

array([310.22, 113.65, 395.69, 299.86, 249.95, 550.82, 238.9 , 273.46,
       413.13, 308.85, 274.86, 334.79, 323.78, 176.56, 464.02, 338.68,
       113.29, 429.09, 176.68, 182.6 , 495.63, 158.03, 246.41, 474.44,
       340.39, 346.37, 413.11, 173.35, 427.5 , 368.62, 182.05, 490.39,
       216.12, 199.88, 528.91, 220.24, 218.66, 302.57, 287.2 , 231.47,
       299.47, 258.08, 219.11, 439.15, 115.15, 466.63, 256.53, 248.95,
       577.39, 245.56, 226.11, 332.07, 325.71, 312.96, 357.57, 156.88,
       438.97, 374.93,  91.56, 425.23, 323.67, 207.71, 576.25, 231.16,
       282.45, 389.96, 341.25, 326.  , 373.  , 371.11, 149.33, 456.22,
       373.87,  66.11, 447.07, 211.57, 206.48, 451.19, 172.36, 178.83,
       323.39, 281.44, 392.41, 339.39, 184.2 , 389.66, 181.12, 483.01,
       256.7 , 197.63, 505.04, 211.65, 206.18, 389.44, 275.64, 285.94,
       313.37, 269.97, 197.6 , 481.62, 437.06, 183.3 , 506.61, 280.51,
       247.11, 509.47, 284.17, 349.67, 219.92, 316.79, 348.56, 321.71,
      

In [192]:
days =np.array([i["day"] for i in cleaned_tickets])

In [193]:
days

array(['Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun', 'Mon', 'Tue', 'Wed',
       'Thu', 'Fri', 'Sat', 'Sun', 'Mon', 'Tue', 'Wed', 'Thu', 'Fri',
       'Sat', 'Sun', 'Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun',
       'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun', 'Mon', 'Tue', 'Wed',
       'Thu', 'Fri', 'Sat', 'Sun', 'Mon', 'Tue', 'Wed', 'Fri', 'Sat',
       'Sun', 'Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun', 'Tue',
       'Wed', 'Thu', 'Fri', 'Sat', 'Sun', 'Mon', 'Tue', 'Wed', 'Thu',
       'Fri', 'Sat', 'Sun', 'Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat',
       'Sun', 'Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun', 'Tue',
       'Wed', 'Thu', 'Fri', 'Sat', 'Mon', 'Tue', 'Wed', 'Thu', 'Fri',
       'Sat', 'Sun', 'Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun',
       'Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun', 'Tue', 'Wed',
       'Thu', 'Fri', 'Sat', 'Sun', 'Mon', 'Tue', 'Wed', 'Thu', 'Fri',
       'Sat', 'Sun', 'Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun',
       'Mon', 'Tue',

In [194]:
mean_price = price.mean()
mean_price

307.45072847682115

In [195]:
std_price = price.std()
std_price

112.1276458434013

In [196]:
unique_days = np.unique(days)
unique_days

array(['Fri', 'Mon', 'Sat', 'Sun', 'Thu', 'Tue', 'Wed'], dtype='<U3')

In [197]:
total_revenue = np.array([price[days == d].sum() for d in unique_days])
total_revenue

array([13723.03,  9937.62, 13884.64, 13449.99, 14246.81, 14080.47,
       13527.56])

In [198]:
ticket_count = np.array([np.sum(days == d) for d in unique_days])
ticket_count

array([45, 33, 45, 44, 45, 45, 45])

In [199]:
for d,tc,tr in  zip(unique_days, ticket_count,total_revenue):
    print(f"{d} : {tc} : {tr}")

Fri : 45 : 13723.029999999999
Mon : 33 : 9937.62
Sat : 45 : 13884.64
Sun : 44 : 13449.990000000002
Thu : 45 : 14246.810000000003
Tue : 45 : 14080.470000000001
Wed : 45 : 13527.56


In [200]:
daily_count = price.sum()
daily_count

92850.12

In [201]:
round(daily_count) == round(total_revenue.sum())

True

# Task 4

In [202]:
threshold = np.percentile(price, 90)

In [203]:
threshold

467.923

In [204]:
high_price = price>=threshold
price[high_price]

array([550.82, 495.63, 474.44, 490.39, 528.91, 577.39, 576.25, 483.01,
       505.04, 481.62, 506.61, 509.47, 515.32, 468.  , 491.38, 494.67,
       533.28, 558.18, 490.33, 469.3 , 512.69, 478.04, 508.34, 568.84,
       502.32, 470.93, 516.25, 495.41, 490.68, 539.53, 486.29])

In [205]:
high_price_count = len(price[high_price])
high_price_count

31

In [206]:
price[high_price]>=threshold

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True])

# Task 5

In [207]:
report = {"total_tickets": len(tickets) , 
          "cleaned_tickets" : cleaned_tickets[1],
           "mean_price": mean_price,
           "std_price": std_price, 
           "daily_totals" : daily_count,
            "high_price_count" : high_price_count 
}

In [208]:
report

{'total_tickets': 320,
 'cleaned_tickets': {'ticket_id': 'T1610-0002',
  'route': 'SFO-SEA',
  'day': 'Wed',
  'days_to_departure': 57,
  'class': 'economy',
  'price_usd': 113.65},
 'mean_price': 307.45072847682115,
 'std_price': 112.1276458434013,
 'daily_totals': 92850.12,
 'high_price_count': 31}

In [209]:
" ".join(f"{n} : {m}"for n,m in report.items())

"total_tickets : 320 cleaned_tickets : {'ticket_id': 'T1610-0002', 'route': 'SFO-SEA', 'day': 'Wed', 'days_to_departure': 57, 'class': 'economy', 'price_usd': 113.65} mean_price : 307.45072847682115 std_price : 112.1276458434013 daily_totals : 92850.12 high_price_count : 31"